# Precondition df

In [1]:
import os
import pandas as pd

In [9]:
ID_arr = [1,2,3,4,5,6]
day_arr = [1,2]
set = [4,5,6]

In [ ]:
col_add = ['subid', 'Thermal', 'Traffic noise levels', 'Speech']
try:
    master_data = pd.read_excel("Experiment_edited_V3.1.xlsx", usecols=['Start Date', 'End Date', 'Experiment set', 'Experiment id', 'Time_thermal']+col_add)
except ValueError:
    master_data = pd.read_excel("Experiment_edited_V3.1b.xlsx", usecols=['Start Date', 'End Date', 'Experiment set', 'Experiment id', 'Time_thermal']+col_add)
new = master_data['Time_thermal'].str.split("-", n=1, expand=True)
master_data['New_Start_Date'] = pd.to_datetime(master_data['Start Date']).dt.date
master_data['New_End_Date'] = pd.to_datetime(master_data['Start Date']).dt.date
master_data['New_Start_Date'] = master_data['New_Start_Date'].astype(str) + ' ' + new[0]
master_data['New_End_Date'] = master_data['New_End_Date'].astype(str) + ' ' + new[1]
master_data['New_Start_Date'] = master_data['New_Start_Date'].astype('datetime64[ms]')
master_data['New_End_Date'] = master_data['New_End_Date'].astype('datetime64[ms]')
master_data

```Additional column experiment_set, experiment_id, thermal, noiselevels, subid, speech```

## HR Creation

In [ ]:
from datetime import datetime, timedelta
for set_ in set:
    with pd.ExcelWriter(f'HR_set{set_}.xlsx', engine="xlsxwriter", datetime_format='yyyy-mm-dd hh:mm:ss.000') as writer:        
        for id_ in ID_arr:
            result = pd.DataFrame(columns=["Time","HR","RR","MS","SKINCONTACT","experiment_set", "experiment_id"]+col_add)
            for day_ in day_arr:
                print(f"process set: {set_} day: {day_} id: {id_}")
                try:
                    csv_arr = os.listdir(f'HR/set{set_}/Day{day_}/ID{id_}/')
                except FileNotFoundError:
                    print(f"Not Found set: {set_} day: {day_} id: {id_}")
                    continue
                csv_arr = csv_arr[:len(csv_arr)//2]
                for i in range(len(csv_arr)):
                    csv_arr[i] = csv_arr[i][-20:-4]

                for csv_ in csv_arr:
                    result_temp = pd.DataFrame(columns=["Time","HR","RR","MS","SKINCONTACT","experiment_set", "experiment_id"]+col_add)
                    file_name_HR = f'HR/set{set_}/Day{day_}/ID{id_}/HR_'+csv_+'.txt'
                    skiprows = 6
                    data_HR = pd.read_csv(file_name_HR, skiprows=skiprows, sep=" ", header=None, names=["CHANNEL","RR","MS","SKINCONTACT"], index_col=False)
                    timestamp = datetime.strptime(csv_, "%d.%m.%Y_%H.%M")
                    result_temp['Time'] = [timestamp]*len(data_HR)
                    result_temp['experiment_set'] = [set_]*len(data_HR)
                    result_temp['experiment_id'] = [id_]*len(data_HR)
                    result_temp['MS'] = pd.to_timedelta(data_HR['MS'],'ms')
                    result_temp['Time'] = result_temp['Time'] + result_temp['MS']
                    result_temp['HR'] = data_HR['CHANNEL']
                    result_temp['RR'] = data_HR['RR']
                    result_temp['SKINCONTACT'] = data_HR['SKINCONTACT']
                    
                    delta = 5
                    max_timestamp = timestamp.replace(hour=23, minute=59, second=59)
                    while True:
                        timestamp = timestamp + timedelta(minutes=delta)
                        
                        sliceData = master_data[(master_data['New_Start_Date']<=timestamp) & (master_data['New_End_Date']>=timestamp) & (master_data['Experiment set']==f'set{set_}') & (master_data['Experiment id'] == f'id{id_}')]
                        
                        if timestamp > max_timestamp:
                            print(f"Slice data not found: {csv_}")
                            break

                        if sliceData.empty:
                            delta+=5
                            continue
                        
                        result_temp[col_add] = [list(sliceData[col_add].values[0])]*len(data_HR)
                        break
                    
                    frames = [result, result_temp]
                    result = pd.concat(frames)
                # break
            result.to_excel(writer, sheet_name=f"id{id_}", index=False)
            # display(result)
            # break
    # break

## ECG Creation

In [ ]:
from datetime import datetime
for set_ in set:
    print(f"process {set_}")
    # with pd.ExcelWriter(f'ECG_set{set_}.xlsx', engine="xlsxwriter", datetime_format='yyyy-mm-dd hh:mm:ss.000') as writer:        
    for id_ in ID_arr:
        result = pd.DataFrame(columns=["Time","ECG","MS","experiment_set", "experiment_id"])
        for day_ in day_arr:
            # print(f"process set: {set_} day: {day_} id: {id_}")
            try:
                csv_arr = os.listdir(f'HR/set{set_}/Day{day_}/ID{id_}/')
            except FileNotFoundError:
                print(f"Not Found set: {set_} day: {day_} id: {id_}")
                continue
            csv_arr = csv_arr[:len(csv_arr)//2]
            for i in range(len(csv_arr)):
                csv_arr[i] = csv_arr[i][-20:-4]

            for csv_ in csv_arr:
                result_temp = pd.DataFrame(columns=["Time","ECG","MS","experiment_set", "experiment_id"])
                file_name_HR = f'HR/set{set_}/Day{day_}/ID{id_}/ECG_'+csv_+'.txt'
                skiprows = 6
                data_ECG = pd.read_csv(file_name_HR, skiprows=skiprows, sep=" ", header=None, names=["CHANNEL","MS"], index_col=False)
                
                # break
                timestamp = datetime.strptime(csv_, "%d.%m.%Y_%H.%M")
                result_temp['Time'] = [timestamp]*len(data_ECG)
                result_temp['experiment_set'] = [set_]*len(data_ECG)
                result_temp['experiment_id'] = [id_]*len(data_ECG)
                result_temp['MS'] = pd.to_timedelta(data_ECG['MS'],'ms')
                result_temp['Time'] = result_temp['Time'] + result_temp['MS']
                result_temp['ECG'] = data_ECG['CHANNEL']
                frames = [result, result_temp]
                result = pd.concat(frames)
            # break
        result.to_csv(f'ECG_set{set_}_id{id_}.csv', index=False)
        # display(result)
        # break
    # break